## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [11]:
import warnings  # Import warnings before using it

get_ipython().system('pip install lightgbm')  # Install LightGBM

warnings.filterwarnings('ignore')  # Suppress warnings

from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder


In [12]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Bayesian_optimization_case_study Unit 18.2.ipynb',
 'Bayesian_optimization_case_study.ipynb',
 'data']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [2]:
def simple_func(a, b):
    return a + b

In [3]:
!pip install bayesian-optimization

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [4]:
from bayes_opt import BayesianOptimization

# Define the function to optimize
def simple_func(a, b):
    return a + b  # Example function

# Initialize the optimizer
optimizer = BayesianOptimization(
    f=simple_func,  # Function to optimize
    pbounds={'a': (1, 3), 'b': (4, 7)}  # Bounds for parameters
)

# Run the optimization
optimizer.maximize()

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 7.878     | 2.488     | 5.39      |
| 2         | 6.559     | 1.542     | 5.017     |
| 3         | 6.261     | 1.539     | 4.722     |
| 4         | 7.756     | 2.855     | 4.901     |
| 5         | 8.959     | 2.608     | 6.351     |
| 6         | 9.953     | 2.955     | 6.999     |
| 7         | 9.224     | 2.224     | 6.999     |
| 8         | 9.955     | 2.991     | 6.965     |
| 9         | 9.935     | 2.937     | 6.998     |
| 10        | 9.947     | 2.948     | 6.999     |
| 11        | 9.946     | 2.992     | 6.954     |
| 12        | 10.0      | 3.0       | 7.0       |
| 13        | 9.972     | 2.973     | 6.999     |
| 14        | 9.972     | 2.973     | 6.999     |
| 15        | 9.972     | 2.973     | 6.999     |
| 16        | 9.972     | 2.973     | 6.999     |
| 17        | 9.972     | 2.973     | 6.999     |
| 18        | 9.967     | 2.984     | 6.983     |


These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [5]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 31        | 8.175     | 2.381     | 5.794     |
| 32        | 6.92      | 2.169     | 4.751     |
| 33        | 6.854     | 1.649     | 5.205     |
| 34        | 6.854     | 1.649     | 5.205     |
| 35        | 6.854     | 1.649     | 5.205     |


Great, now let's print the best parameters and the associated maximized target.

In [6]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [17]:
#import pandas as pd

# Load the dataset
#train_df = pd.read_csv('flight_delays_train.csv.zip')
#test_df = pd.read_csv('flight_delays_test.csv.zip')


import pandas as pd
train_df = pd.read_csv('C:/Users/samia/OneDrive/Documents/GitHub/springboard/1720450277_Bayesian_Optimization/18.2.6 - Bayesian Optimization/data/flight_delays_train.csv')
test_df = pd.read_csv('C:/Users/samia/OneDrive/Documents/GitHub/springboard/1720450277_Bayesian_Optimization/18.2.6 - Bayesian Optimization/data/flight_delays_test.csv')


**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [18]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [19]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [20]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [23]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


Apply the earlier defined feature engineering functions to the full dataframe.

In [30]:
# Feature engineering function (updated to include the 'flightUC' column creation)
def feature_eng(df):
    # Assuming 'flight' and 'UniqueCarrier' exist in df
    df['flightUC'] = df['flight'].astype(str) + df['UniqueCarrier'].astype(str)  # Combining 'flight' and 'UniqueCarrier'

    # Rest of your feature engineering steps...
    
    return df

# Now apply feature engineering and label encoding
full_df = feature_eng(full_df)

# Apply label encoding to the relevant columns
from sklearn.preprocessing import LabelEncoder

def label_enc(df_column):
    return LabelEncoder().fit_transform(df_column)

# Apply encoding for all specified columns
for column in ['UniqueCarrier', 'Origin', 'Dest', 'flight', 'flightUC', 'DestUC', 'OriginUC']:
    if column in full_df.columns:  # Check if the column exists before applying label encoding
        full_df[column] = label_enc(full_df[column])
    else:
        print(f"Warning: Column '{column}' not found in the DataFrame")




Split the new full dataframe into X_train and X_test. 

In [31]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [32]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [33]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [58]:
import lightgbm as lgb
from lightgbm import early_stopping
from bayes_opt import BayesianOptimization

# Check if categorical features exist in the dataset
categorical_features = ['Dest', 'carrier_count_per month']  
for feature in categorical_features:
    if feature not in X_train.columns:
        raise ValueError(f"Feature '{feature}' not found in training data.")

# Convert columns to 'category' dtype (if they aren't already)
for feature in categorical_features:
    if X_train[feature].dtype != 'category':
        X_train[feature] = X_train[feature].astype('category')
    if X_val[feature].dtype != 'category':
        X_val[feature] = X_val[feature].astype('category')

# Define the parameters for the LightGBM model
params = {
    'objective': 'binary',         # Binary classification task
    'metric': 'auc',               # Evaluate using AUC
    'boosting_type': 'gbdt',       # Gradient boosting decision tree
    'num_leaves': 31,              # Number of leaves in one tree
    'learning_rate': 0.05,         # Learning rate
    'feature_fraction': 0.9,       # Fraction of features to use for each tree
    'lambda_l2': 0.1              # L2 regularization
}

# Prepare the training and validation datasets
lgtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgval = lgb.Dataset(X_val, y_val, categorical_feature=categorical_features)

# Early stopping callback
callbacks = [early_stopping(stopping_rounds=100)]

# Train the model
model = lgb.train(params,
                  lgtrain,
                  num_boost_round=1000,
                  valid_sets=[lgtrain, lgval],
                  valid_names=['train', 'valid'],
                  callbacks=callbacks)  # Using callbacks for early stopping

# Return the best AUC score on the validation set
print(f"Best AUC score: {model.best_score['valid']['auc']}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7841, number of negative: 33119
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006911 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2758
[LightGBM] [Info] Number of data points in the train set: 40960, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191431 -> initscore=-1.440741
[LightGBM] [Info] Start training from score -1.440741
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[207]	train's auc: 0.899166	valid's auc: 0.71718
Best AUC score: 0.7171802731344172


 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [41]:
lgbBO.max

Review the process at each step by using the '.res[0]' function.

In [49]:
print(X_train.columns)


Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance', 'flight', 'begin_of_month',
       'midddle_of_month', 'end_of_month', 'hour', 'morning', 'day', 'evening',
       'night', 'winter', 'spring', 'summer', 'autumn', 'holiday', 'weekday',
       'airport_dest_per_month', 'airport_origin_per_month',
       'airport_dest_count', 'airport_origin_count', 'carrier_count',
       'carrier_count_per month', 'flightUC'],
      dtype='object')


In [55]:
print(X_train.columns)


Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance', 'flight', 'begin_of_month',
       'midddle_of_month', 'end_of_month', 'hour', 'morning', 'day', 'evening',
       'night', 'winter', 'spring', 'summer', 'autumn', 'holiday', 'weekday',
       'airport_dest_per_month', 'airport_origin_per_month',
       'airport_dest_count', 'airport_origin_count', 'carrier_count',
       'carrier_count_per month', 'flightUC'],
      dtype='object')


In [56]:
import lightgbm as lgb
from lightgbm import early_stopping

# Check if categorical features exist in the dataset
categorical_features = ['Dest', 'carrier_count_per month']  # Replace with actual column names
for feature in categorical_features:
    if feature not in X_train.columns:
        raise ValueError(f"Feature '{feature}' not found in training data.")

# Convert columns to 'category' dtype (if they aren't already)
for feature in categorical_features:
    if X_train[feature].dtype != 'category':
        X_train[feature] = X_train[feature].astype('category')
    if X_val[feature].dtype != 'category':
        X_val[feature] = X_val[feature].astype('category')

# Define the parameters for the LightGBM model
params = {
    'objective': 'binary',         # Binary classification task
    'metric': 'auc',               # Evaluate using AUC
    'boosting_type': 'gbdt',       # Gradient boosting decision tree
    'num_leaves': 31,              # Number of leaves in one tree
    'learning_rate': 0.05,         # Learning rate
    'feature_fraction': 0.9,       # Fraction of features to use for each tree
    'lambda_l2': 0.1              # L2 regularization
}

# Prepare the training and validation datasets
lgtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgval = lgb.Dataset(X_val, y_val, categorical_feature=categorical_features)

# Train the model with early stopping via callbacks
callbacks = [early_stopping(stopping_rounds=100)]

model = lgb.train(params,
                  lgtrain,
                  num_boost_round=1000,
                  valid_sets=[lgtrain, lgval],
                  valid_names=['train', 'valid'],
                  callbacks=callbacks)  # Early stopping via callbacks

# Return the best score on the validation set
print(f"Best AUC score: {model.best_score['valid']['auc']}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7841, number of negative: 33119
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2758
[LightGBM] [Info] Number of data points in the train set: 40960, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191431 -> initscore=-1.440741
[LightGBM] [Info] Start training from score -1.440741
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[207]	train's auc: 0.899166	valid's auc: 0.71718
Best AUC score: 0.7171802731344172
